Normal modelimize alternatif olarak oluşturduğumuz bir modeldir. Normal modelimizin aksine burada girdinin doküman türünü sınıflandırıyoruz. Bunun sonucu olarak yalnızca girilen dokümanın türünde çıktılar alınıyor.

Girdilerin (CSV değerlendirme dosyasının) özelliklerine bağlı olarak 2 modelimizin performansı farklılık gösterebileceğinden dolayı duruma göre birini seçeceğiz.

In [1]:
# Yüklü değillerse gerekli kütüphanelerin yüklenmesi

#!pip install torch
#!pip install transformers

In [2]:
# Gerekli kütüphanelerin içe aktarılması

from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch

In [3]:
# Cümle temsillerinin deserialize edilerek RAM'e alınması

import pickle

embeddings = []

for i in range(0, 501):
    with open(f'embeddings/sbert_embeddings{i}.json', 'rb') as file:
        obj = pickle.load(file)
        embeddings.extend(obj)

In [4]:
# Huggingface servisinde barındırılan BERT modelimizin indirilmesi

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sfurkan/LexBERT-turkish-uncased')
model.max_seq_length = 65536

No sentence-transformers model found with name C:\Users\Furkan/.cache\torch\sentence_transformers\sfurkan_LexBERT-turkish-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Furkan/.cache\torch\sentence_transformers\sfurkan_LexBERT-turkish-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification 

In [5]:
# Veri kümemizin içe aktarılması

TARGETED_DATASET_PATH = r'data.csv'

texts = pd.read_csv(TARGETED_DATASET_PATH)

In [6]:
# Herhangi bir cümlenin indexini alıp içinde bulunduğu dokümanın indexini döndüren dictionary veri yapısının oluşturulması

from tqdm import tqdm

sentenceIndex2DocIndex = {}
lastSentenceIndex = 0
currentDocIndex = 0
for text in tqdm(texts.iloc):
    text = text['data_text']
    sentences = text.strip().split('.')
    
    for idx, sentence in enumerate(sentences):
        sentenceIndex2DocIndex[idx + lastSentenceIndex] = currentDocIndex
    
    lastSentenceIndex += len(sentences)
    currentDocIndex += 1

4142it [00:01, 2694.50it/s]


In [7]:
# Doküman türünü çıkarabilen 'text classification' modelimizin indirilmesi

from transformers import pipeline

classificationPipeline = pipeline("text-classification", model="sfurkan/LexBERT-textclassification-turkish-uncased", tokenizer = "sfurkan/LexBERT-textclassification-turkish-uncased")

In [8]:
# Girilen dokümanın BERT tarafından kullanılabilmesi için ilk 512 kelimeyi alacak şekilde kırpılması. (Yalnızca 'text classification' için kullanılır)

def cropDocument(document, cropLimit=512):
    totalWords = []
    sentences = document.replace('\n', '. ').split('.')
    for sent in sentences:
        sent = sent.strip()
        words = sent.split(' ')
        for word in words[:-1]:
            if len(totalWords) >= cropLimit:
                return totalWords
            totalWords.append(word)
        totalWords.append(f"{words[-1]}.")
    return ' '.join(totalWords)

In [9]:
# Girilen dokümanın türünü döndürecek fonksiyon

def getDocumentType(inputText):
    return classificationPipeline(cropDocument(inputText))[0]["label"]

In [10]:
# Doküman içerisindeki cümle temsillerinin organizasyonunun 'liste içinde liste' şeklinde bir veri yapısında sağlanması
documentEmbeddings = []
def initializeDocEmbeddings():
    embeddingIndex = 0
    for idx in tqdm(range(texts.shape[0])):
        text = texts.loc[idx]["data_text"]
        sents = text.strip().split('.')
        sentEmbeddings = []
        for sent in sents:
            sentEmbeddings.append(embeddings[embeddingIndex])
            embeddingIndex += 1
        documentEmbeddings.append(sentEmbeddings)
initializeDocEmbeddings()

100%|████████████████████████████████████████████████████████████████████████████| 4142/4142 [00:02<00:00, 1699.64it/s]


In [11]:
# Verilen girdi dokümanı kullanarak benzer dokümanları döndüren fonksiyon

from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

def getSimilarDocuments(inputText, topk=10):
    # Girdinin doküman türünün belirlenmesi
    docType = getDocumentType(inputText)

    inputEmbedding = model.encode(inputText)

    documentScores = []    
    for idx, docEmb in enumerate(documentEmbeddings):
        # Iterate edilen dokümanın kategorisi, aranan kategori değilse atlanması
        if texts.iloc[idx]["kategori"] != docType:
            continue
        
        similarities = cosine_similarity(docEmb, [inputEmbedding])
        meanSim = 0
        for sim in similarities:
            meanSim += sim[0]
        meanSim /= len(similarities)
        documentScores.append((idx, meanSim))

    unfilteredDocuments = []
    maxSim = -1
    for idx, docScore in documentScores:
        if docScore > maxSim:
            maxSim = docScore
            unfilteredDocuments.append((texts.iloc[idx], docScore))
    
    documentsToReturn = sorted(unfilteredDocuments, key=lambda x: x[1], reverse=True)[:min(len(unfilteredDocuments), topk)]
    
    documentRows = []
    for document, score in documentsToReturn:
        documentRows.append((*document.values, score))
        
        #dfRows.append(pd.DataFrame([document, pd.Series(score)], columns=texts.columns))
    similarDF = pd.DataFrame(documentRows, columns=[*texts.columns, 'Similarity Score'])
    
    return similarDF

In [13]:
# Örnek kullanım

INPUT_TEXT = "Elektronik eşya vergilendirmesi hakkında kanun"

getSimilarDocuments(INPUT_TEXT, topk=5)

,id,kategori,baslik,rega_no,mukerrer_no,rega_tarihi,kurum,mevzuat_no,belge_sayi,mevzuat_tarihi,donem,sira_no,madde_sayisi,data_text,url,kanunum_url,Similarity Score
0,129,Kanun,YATIRIM ORTAMININ İYİLEŞTİRİLMESİ AMACIYLA BAZ...,29796,0.0,2016-08-09,NaN,6728,NaN,2016-07-15,NaN,NaN,78,Resmî Gazete Tarihi: 09.08.2016 Resmî Gazete S...,http://www.mevzuat.gov.tr/Metin.Aspx?MevzuatKo...,https://www.kanunum.com/content/2654308,0.703920
1,29,Kanun,GELİR VERGİSİ KANUNU İLE BAZI KANUNLARDA DEĞİŞ...,30836,1.0,2019-07-19,NaN,7186,NaN,2019-07-17,NaN,NaN,35,Resmî Gazete Tarihi: 19.07.2019 Resmî Gazete S...,http://www.resmigazete.gov.tr/eskiler/2019/07/...,https://www.kanunum.com/content/8613190,0.699733
2,9,Kanun,İŞ KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KAR...,29116,1.0,2014-09-11,NaN,6552,NaN,2014-09-10,NaN,NaN,149,Resmî Gazete Tarihi: 11.09.2014 Resmî Gazete S...,https://www.mevzuat.gov.tr/mevzuat?MevzuatNo=6...,https://www.kanunum.com/content/1233870,0.685094
3,5,Kanun,SAĞLIK BAKANLIĞI VE BAĞLI KURULUŞLARININ TEŞKİ...,28886,0.0,2014-01-18,NaN,6514,NaN,2014-01-02,NaN,NaN,56,Resmî Gazete Tarihi: 18.01.2014 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2014/01...,https://www.kanunum.com/content/942558,0.674757
4,3,Kanun,BÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İL...,28350,0.0,2012-07-11,NaN,6338,NaN,2012-06-29,NaN,NaN,19,Resmî Gazete Tarihi: 11.07.2012 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2012/07...,https://www.kanunum.com/content/344634,0.668466
